In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import os
import pickle
import matplotlib.pyplot as plt
import statsmodels.api as sm

/home/harshit/anaconda3/lib/python3.6/site-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [122]:
mod_path="/home/harshit/Project/complete_models/2014/"
lis = pd.DataFrame(os.listdir(mod_path))
coeff = pd.DataFrame(columns = ["Symbol","const","SMB %", "HML %","Rm-Rf %", "high-low"] )
#Making a dataframe of all coffecients
for i in lis[0]:
    data = open(mod_path+i,"rb")
    symbol = i.split('.')[0]
    model = pickle.load(data)
    frame = pd.DataFrame(model.params).T
    frame.insert(0,"Symbol",symbol)
    coeff = coeff.append(frame,ignore_index=True)

In [123]:
# Making dataframe of Mean Returns
ret_path = "/home/harshit/Project/risk_free_ret/2014/"
lis = pd.DataFrame(os.listdir(ret_path))
returns = pd.DataFrame(columns = ["Symbol","mean_ret"])
factors = pd.read_csv("/home/harshit/Project/allfactors.csv")
for i in lis[0]:
    data = pd.read_csv(ret_path+i)
    data_factors = pd.merge(data,factors,on="Date")
    symbol = i.split(".")[0]
    mean = data_factors.Return.mean()
    df = pd.DataFrame([[symbol,mean]],columns = ["Symbol","mean_ret"])
    returns = returns.append(df,ignore_index=True)
    factor =data_factors[["SMB %","HML %","WML %","Rm-Rf %","high-low"]]
    factors_mean = pd.DataFrame(factor.mean(),columns = ["Mean"])


In [124]:
all_data = pd.merge(returns,coeff,on="Symbol", left_index=True, right_index=True) #dataframe of all mean returns and coeff

In [125]:
X = all_data[["const","SMB %","HML %","WML %","Rm-Rf %","high-low"]]
y = all_data[["mean_ret"]]

model1 = sm.OLS(y,X).fit()

In [126]:
X = all_data[["const","Rm-Rf %"]]
y = all_data[["mean_ret"]]

model2 = sm.OLS(y,X).fit()

In [127]:
X = all_data[["const","high-low"]]
y = all_data[["mean_ret"]]

model3 = sm.OLS(y,X).fit()

In [128]:
X = all_data[["SMB %","HML %","WML %","Rm-Rf %","high-low"]]
y = all_data[["mean_ret"]]

model4 = sm.OLS(y,X).fit()

In [129]:
model_values = pd.DataFrame(model1.params,columns=["Model"]) #comparing means to regression coeffecients
#comparison = pd.merge(model_values,factors_mean)
factors_mean.rename(columns={'Mean':"Actual"}, inplace=True)
comparison=pd.merge(model_values,factors_mean,left_index=True,right_index=True)
comparison.to_csv("/home/harshit/Project/fama_mcbeth/"+"2014.csv")

In [130]:
comparison

,Model,Actual
SMB %,0.031241,0.098350
HML %,0.073954,0.023263
WML %,0.035391,0.012303
Rm-Rf %,0.158634,0.109352
high-low,-0.119207,-0.206152


In [131]:
model_values

,Model
const,0.136284
SMB %,0.031241
HML %,0.073954
WML %,0.035391
Rm-Rf %,0.158634
high-low,-0.119207


In [35]:
data_factors

,Date,Return,SMB %,HML %,WML %,Rm %,Rf %,Rm-Rf %,high,low,high-low
0,20170103,-0.774669,0.598777,0.323179,0.543813,0.567004,0.024328,0.542676,-0.561882,-0.303657,-0.258224
1,20170104,-0.633671,1.007015,-0.159444,-1.089314,0.095347,0.024328,0.071019,0.292570,-0.230464,0.523034
2,20170105,-3.186880,-0.386523,1.551816,0.649860,1.073165,0.024027,1.049138,-0.830148,0.045865,-0.876012
3,20170106,1.345836,-0.337343,0.383829,0.433529,-0.310864,0.024027,-0.334890,0.285493,-0.190515,0.476008
4,20170109,0.438936,0.630020,0.311275,-0.103015,0.062479,0.024027,0.038452,-0.245151,0.552532,-0.797683
5,20170110,0.308199,0.264602,-0.118204,0.265090,0.626342,0.024027,0.602315,-0.758949,-0.207711,-0.551238
6,20170111,2.642640,-0.857073,1.640761,0.686944,1.087354,0.024027,1.063327,-1.291578,0.667705,-1.959283
7,20170112,-5.640766,-0.972456,0.390246,0.357710,0.223587,0.024328,0.199259,-0.131327,0.914867,-1.046194
8,20170113,-2.164406,0.390916,0.013200,0.186406,-0.150544,0.024328,-0.174872,0.503656,-0.986266,1.489922
9,20170116,-4.214804,0.022881,0.819896,0.370514,0.266657,0.024328,0.242329,-0.169703,-0.431808,0.262106


In [30]:
mean

0.02021581664122145